In [3]:
import pandas as pd
import numpy as np
import requests
from datetime import datetime


pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)


### FUNÇÃO PARA PEGAR TOKEN DE AUTENTICAÇÃO NA API
def get_token(email,senha):
    body = {"email": email,"password": senha}
    r = requests.post('https://api.oplab.com.br/v3/domain/users/authenticate',json=body)
    return r.json()['access-token']

### FUNÇÃO PARA COLETAR LISTA DE AÇÕES COM MAIOR LIQUIDEZ EM OPÇÕES
def pegar_mais_liqs(token,rank = 20):
    lista = []
    r = requests.get('https://api.oplab.com.br/v3/market/statistics/realtime/highest_options_volume?order_by=put&limit={}'.format(rank),headers={"Access-Token": token}).json()
    for i in r:
        lista.append(i['symbol'])
    return lista

### FUNÇÃO PARA COLETAR TODAS AS OPÇÕES DISPONÍVEIS POR ATIVO
def get_all_options(symbol,token):
    r = requests.get('https://api.oplab.com.br/v3/market/options/{}'.format(symbol), headers={"Access-Token": token})
    return r.json()

### FUNÇÃO PARA ORGANIZAR OPÇÕES EM UM DATAFRAME
def ofertas_opcoes(ativo,token,ask_dif_zero = 1):
    ### USAR ask_dif_zero = 1 PARA BUSCAR APENAS OPÇÕES COM OFERTA DE VENDA DISPONÍVEL
    symbol = []
    strike = []
    bid = []
    ask = []
    moneyness = []
    ativo_obj = []
    tipo = []
    vcto = []
    spot_price = []
    contract_size = []
    all_options = get_all_options(ativo,token)
    for i in range(len(all_options)):
        maturity = (datetime.strptime(all_options[i]['due_date'],"%Y-%m-%d")-datetime.today())
        if all_options[i]['strike'] >0 and (all_options[i]['ask'] > 0) == ask_dif_zero:
            p_spot = 0 if all_options[i]['spot_price'] == 0 else all_options[i]['spot_price']
            mnns = 0 if all_options[i]['spot_price'] == 0 else all_options[i]['strike'] / all_options[i]['spot_price'] -1 
            symbol.append(all_options[i]['symbol'])
            tipo.append(all_options[i]['type'])
            strike.append(all_options[i]['strike'])
            bid.append(all_options[i]['bid'])
            ask.append(all_options[i]['ask'])
            moneyness.append(round(mnns*100,4))
            ativo_obj.append(ativo)
            vcto.append(maturity.days)
            spot_price.append(p_spot)
            contract_size.append(all_options[i]['contract_size'])

    ofertas = pd.DataFrame({'OBJ':ativo_obj,'OPÇÃO':symbol,'TIPO':tipo,'SPOT':spot_price,'K':strike,'BID':bid,'ASK':ask,'DIST_ATM':moneyness,'VCTO':vcto,'SIZE':contract_size})
    return ofertas

### INSERIR EMAIL E SENHA --> get_token(EMAIL,SENHA)
try:
    token = get_token('mnchamma@hotmail.com','LIb321*54') 
except:
    print('TOKEN ERRADO')

### COLETAR LISTA DE 20 ATIVOS COM MAIOR LIQUIDEZ EM OPÇÕES
lista_ativos = pegar_mais_liqs(token,20)

### FILTRAR OPÇÕES COM VENCIMENTO MAIS LONGOS
for ativo in lista_ativos:
    ### SE O ATIVO FOR DIFERENTE DE IBOV
    if ativo == 'IBOV':
        continue
    ofertas = ofertas_opcoes(ativo,token)
    vencimentos = []
    for x in list(ofertas['VCTO']):
        if x not in vencimentos:
            vencimentos.append(x)
    vencimentos.sort()
    if len(vencimentos) <= 1:
        continue
    print(ofertas[ofertas['VCTO'] >= vencimentos[-2]])




#

       OBJ     OPÇÃO  TIPO   SPOT      K  BID   ASK  DIST_ATM  VCTO  SIZE
783  PETR4  PETRU322   PUT  24.72  28.99  0.0  4.75   17.2735   650   100
784  PETR4  PETRU420   PUT  24.68  38.74  9.0  9.80   56.9692   650   100
785  PETR4  PETRI440  CALL  24.84  40.74  0.0  2.90   64.0097   650   100
786  PETR4   PETRJ33  CALL  24.72  30.24  4.5  5.41   22.3301   678   100
       OBJ     OPÇÃO  TIPO   SPOT      K   BID    ASK  DIST_ATM  VCTO  SIZE
506  VALE3  VALET600   PUT  87.77   60.0  1.53   4.00  -31.6395   615   100
507  VALE3  VALET700   PUT  89.09   70.0  2.83   6.00  -21.4278   615   100
508  VALE3  VALEH166  CALL   0.00  166.0  0.00   3.80    0.0000   615   100
509  VALE3  VALEV830   PUT   0.00   83.0  0.00  15.00    0.0000   678   100
510  VALE3  VALEJ166  CALL   0.00  166.0  0.00   5.62    0.0000   678   100
       OBJ     OPÇÃO  TIPO  SPOT     K   BID    ASK  DIST_ATM  VCTO  SIZE
135  AMER3  AMERC200  CALL  8.59  20.0  0.21   1.50  132.8289   461   100
136  AMER3  AMERC450  CALL